## Running Iterative Clustering
This jupyter notebook can be used to run iterative clustering on large data sets

### Before you get started
#### Harddrive space
This runs best on a machine with a large local ssd. Here is an example of the storage space you'll need
* 250GB - raw_counts.h5ad
* 250GB - normalized.h5ad (if you already have this file, you don't need the raw_counts.h5ad)
* 250GB - space used by iterative clustering to store temporary files

#### Running Jupyter Notebook through ssh
You can run jupyter notebook through SSH! 
1) ssh into the remote machine you want to run it on
2) activate your transcriptomic clustering environment
3) run `nohup jupyter notebook -no-browser --port=1234 &`. This will start up the jupyter notebook server and prevent it from ending if your SSH connection gets terminated. Alternatively, you could use programs like `tmux` or `screen`. Copy the address it shows for step 5
4) back on your machine, run `ssh -NL 1234:localhost:1234 username@remote-machine` to open an SSH tunnel
5) enter the address from step 4 (should be something like 'http://localhost:1234/?token=8d186032bbbe095b294789e863b065a546fcc15b68683c99' Now you should be able to interact with the notebook on the remote machine!

#### Temporary Directories
Because AnnData doesn't support multiple views of filebacked data (e.g. subset=adata[1:5], subset[1:2]), we have to create temporary files for each cluster until we can store the whole cluster into memory. We store these files in the temporary directory - it will mostly cleanup after itself, but always check and remove old tmp files to keep your harddrive free

#### Run normalized data first
and save it so you don't need to rerun if you restart iterative clustering. It takes 60-90 minutes to normalize the data, but you don't need to repeat it if it's already been created. Just start directly at iterative clustering.

#### If SSH disconnects
Just reopen the SSH Tunnel (step 4 and 5), and you'll be able to see the notebook. However, due to a known unresolved issue, when you reopen the notebook you will no longer get log messages from iter_clust. As a work around, you can monitor progress with commands like `top`, `ls` the temporary directory for new files being created, or you can try [this workaround](https://github.com/jupyter/jupyter/issues/83#issuecomment-622984009)

In [ ]:
import tempfile
import os
import shutil
import json
import logging

import matplotlib.pyplot as plt

import numpy as np
import scipy as scp
import scanpy as sc
import pandas as pd
import transcriptomic_clustering as tc
from transcriptomic_clustering.iterative_clustering import (
    build_cluster_dict, iter_clust, OnestepKwargs, summarize_final_clusters
)

In [ ]:
# Setup input/output files
output_file = os.path.expanduser('clusters.json')

path_to_adata = './data/tasic2016counts_sparse.h5ad'
adata = sc.read_h5ad(path_to_adata, backed='r')

logfile = os.path.expanduser('clustering.log')

In [ ]:
%config Application.log_level='INFO'
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Optional for adding a log file
fh = logging.FileHandler(logfile)
fh.setLevel(logging.INFO)
logger.addHandler(fh)

In [ ]:
# Set memory params
tc.memory.set_memory_limit(GB=1)
tc.memory.allow_chunking = True

In [ ]:
# Load Known Modes
known_modes_df = pd.DataFrame(np.random.rand(adata.X.shape[0]), index=adata.obs.index) # fake data as an example

In [ ]:
# Assign kwargs. Any unassigned args will be set to their respective function defaults
means_vars_kwargs = {
    'low_thresh': 1,
    'min_cells': 4
}
highly_variable_kwargs = {
    'max_genes': 3000
}

pca_kwargs = {
    'cell_select': 500000,
    'n_comps': 50,
    'svd_solver': 'randomized'
}

filter_pcs_kwargs = {
    'known_components': None,
    'similarity_threshold': 0.7,
    'method': 'zscore', # or elbow
    'zth': 2,
    'max_pcs': None,
}

# project_kwargs = {
    
# }

# Leave empty if you don't want to use known_modes
filter_known_modes_kwargs = {
    'known_modes': known_modes_df,
    'similarity_threshold': 0.7
}

cluster_louvain_kwargs = {
    'k': 15, # number of nn
    'nn_measure': 'euclidean',
    'knn_method': 'annoy',
    'louvain_method': 'vtraag',
    'weighting_method': 'jaccard',
    'n_jobs': 8, # cpus
    'resolution': 1., # resolution of louvain for taynaud method
}

merge_clusters_kwargs = {
    'thresholds': {
        'q1_thresh': 0.5,
        'q2_thresh': None,
        'cluster_size_thresh': 15,
        'qdiff_thresh': 0.7,
        'padj_thresh': 0.05,
        'lfc_thresh': 1.0,
        'score_thresh': 200,
        'low_thresh': 1
    },
    'k': 2, # number of nn for de merge 
    'de_method': 'ebayes'
}

onestep_kwargs = OnestepKwargs(
    means_vars_kwargs = means_vars_kwargs,
    highly_variable_kwargs = highly_variable_kwargs,
    pca_kwargs = pca_kwargs,
    filter_pcs_kwargs = filter_pcs_kwargs,
#     project_kwargs = project_kwargs,
    filter_known_modes_kwargs = filter_known_modes_kwargs,
    cluster_louvain_kwargs = cluster_louvain_kwargs,
    merge_clusters_kwargs = merge_clusters_kwargs
)

In [ ]:
# Remove old tmp_dir and make new one
try:
    shutil.rmtree(tmp_dir)
except NameError as e:
    pass # tmp_dir didn't exist
tmp_dir = tempfile.mkdtemp()

In [ ]:
# normalize adata
norm_adata_path = os.path.join(tmp_dir, 'normalized.h5ad')
normalized_adata = tc.normalize(adata,copy_to=norm_adata_path)

In [ ]:
normalized_adata.var_names_make_unique()
print(f'{normalized_adata.n_obs} cells')
print(f'{normalized_adata.n_vars} genes')

In [ ]:
# Run clustering
clusters = iter_clust(
    normalized_adata,
    min_samples=4,
    onestep_kwargs=onestep_kwargs,
    random_seed=123,
    tmp_dir=tmp_dir
)

In [ ]:
print(f'final cluster sizes: {[len(cluster) for cluster in clusters]}')

In [ ]:
de_table, linkage, labels = summarize_final_clusters(
    normalized_adata,
    clusters,
    de_thresholds=onestep_kwargs.merge_clusters_kwargs['thresholds'],
    low_th=1,
    de_method='ebayes'
)

In [ ]:
n_markers = 50
marker_genes = de_table['up_genes'][:n_markers] + de_table['down_genes'][:n_markers]
marker_genes.index.names =['cluster_a', 'cluster_b']
marker_genes = marker_genes.to_frame('marker_genes')
print(marker_genes)


In [ ]:
%matplotlib inline

plt.figure()
dn = scp.cluster.hierarchy.dendrogram(linkage, labels=labels)
plt.show

In [ ]:
cluster_dict = build_cluster_dict(clusters)
with open(output_file, 'w') as f:
    json.dump(cluster_dict, f)

In [ ]:
shutil.rmtree(tmp_dir)